# ccdproc-06:   Flatfielding

## Overview
1. Using master FLAT to correct from spatial variation of sensitivity all the images

<pre>
Máster en Astrofísica UCM  -- Técnicas Experimentales en Astrofísica  
Jaime Zamorano and Nicolás Cardiel

This notebook has reproduced parts of the the astropy ccdproc docs: 
https://ccdproc.readthedocs.io/en/latest/reduction_toolbox.html#subtract-bias-and-dark

Version 1.0 2021/01/16  
</pre>

Note that the ``astropy`` package should be installed. In this sense, have a look to the
astropy installation description: https://docs.astropy.org/en/stable/install.html.

In [1]:
from pathlib import Path
import datetime

import matplotlib.pyplot as plt
#import matplotlib.mlab as ml
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable

from astropy.io import fits
from astropy import units as u
from astropy.nddata import CCDData

import ccdproc

In [2]:
plt.style.use('./tea.mplstyle')

### Working with images in a directory
We will use the files of the first night of observations at NOT (Nordic Optical Telescope) 2008 that can be downloaded from   ftp://astrax.fis.ucm.es/pub/users/jaz/NOT_2008_04_12-14/N1/
or 
http://guaix.fis.ucm.es/~jaz/master_TEA/observaciones_NOT_2008/N1/

``directory`` should point to the working directory.  


At this step we have already created a combination of FLAT frames with the result of a several master FLATs (one for each filter used) that we called flat_N1_49.fits, flat_N1_76.fits and flat_N1_78.fits. This file should be in our working directory that also contents the trimmed images.  

In [3]:
from ccdproc import ImageFileCollection
from ccdproc.utils.sample_directory import sample_directory_with_files

dirname = 'N1'   # change to the path of your own working directory
directory = Path(dirname)

Let\'s create a list containing all the FITS files in that directory.  
We asume that the images were already trimmed and BIAS corrected and their names begin with 'zt_' (see previous notebooks 'ccdproc_02' and 'ccdproc_03')

In [4]:
filelist = list(sorted(directory.glob('zt_*.fits')))
print(filelist[10:15])    # printing only from 10 to 20
print(filelist[100:105])    # printing only from 10 to 20

[PosixPath('N1/zt_ALrd120011.fits'), PosixPath('N1/zt_ALrd120012.fits'), PosixPath('N1/zt_ALrd120013.fits'), PosixPath('N1/zt_ALrd120014.fits'), PosixPath('N1/zt_ALrd120015.fits')]
[PosixPath('N1/zt_ALrd120102.fits'), PosixPath('N1/zt_ALrd120103.fits'), PosixPath('N1/zt_ALrd120104.fits'), PosixPath('N1/zt_ALrd120105.fits'), PosixPath('N1/zt_ALrd120106.fits')]


### Reading the master  FLATs files 

In [5]:
master_flat_49 = CCDData.read(directory / 'flat_N1_49.fits')
master_flat_76 = CCDData.read(directory / 'flat_N1_76.fits')
master_flat_78 = CCDData.read(directory / 'flat_N1_78.fits')

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


## Correcting all files from FLAT

### Using ccdproc_correct_flat

In [6]:
keys = ['OBJECT' , 'EXPTIME' , 'ALFLTID' , 'FAFLTID' , 'FBFLTID']
ic1 = ImageFileCollection(directory, keywords=keys, glob_include='zt_*.fits') # only keep track of keys
ic1.summary.colnames

['file', 'OBJECT', 'EXPTIME', 'ALFLTID', 'FAFLTID', 'FBFLTID']

In [7]:
matches = (ic1.summary['FBFLTID'] == 78)
sci_list_78 = ic1.summary['file'][matches]
print(sci_list_78)

       file       
------------------
zt_ALrd120021.fits
zt_ALrd120023.fits
zt_ALrd120024.fits
zt_ALrd120025.fits
zt_ALrd120026.fits
zt_ALrd120027.fits
zt_ALrd120028.fits
zt_ALrd120029.fits
zt_ALrd120030.fits
zt_ALrd120031.fits
               ...
zt_ALrd120087.fits
zt_ALrd120098.fits
zt_ALrd120099.fits
zt_ALrd120100.fits
zt_ALrd120113.fits
zt_ALrd120114.fits
zt_ALrd120115.fits
zt_ALrd120119.fits
zt_ALrd120120.fits
zt_ALrd120121.fits
zt_ALrd120122.fits
Length = 35 rows


In [9]:
sci_images_78 = []
for file in sci_list_78:
    sci_images_78.append(CCDData.read(directory / file)) 

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.


In [10]:
print('Filename          Object        exp Mean std min  max')
exposure = []
for img in sci_images_78:
    print(img.header['FILENAME'], 
          img.header['OBJECT'], 
          img.header['EXPTIME'], 
          int(np.mean(img)), 
          int(np.std(img)), 
          np.min(img), 
          np.max(img))

Filename          Object        exp Mean std min  max
zt_ALrd120021.fits Dome Flat NB#78 10.0 6753 2509 -8.5 16666.0
zt_ALrd120023.fits Dome Flat NB#78 30.0 20303 7538 11.5 29560.5
zt_ALrd120024.fits Dome Flat NB#78 30.0 20363 7560 11.5 29627.0
zt_ALrd120025.fits Dome Flat NB#78 30.0 20356 7557 21.5 29766.0
zt_ALrd120026.fits Dome Flat NB#78 30.0 20313 7541 20.5 29711.0
zt_ALrd120027.fits Dome Flat NB#78 30.0 20242 7514 16.5 33439.5
zt_ALrd120028.fits Dome Flat NB#78 30.0 20229 7509 16.5 41590.0
zt_ALrd120029.fits Dome Flat NB#78 30.0 20227 7509 27.5 30235.5
zt_ALrd120030.fits Dome Flat NB#78 30.0 20333 7548 16.5 37827.5
zt_ALrd120031.fits Dome Flat NB#78 30.0 20320 7543 12.5 29711.5
zt_ALrd120032.fits Dome Flat NB#78 30.0 20320 7543 16.5 31909.0
zt_ALrd120044.fits Sky Flat evening 0.1 2693 962 -29.0 6723.0
zt_ALrd120045.fits Sky Flat evening 1.199 8912 3405 13.5 65182.0
zt_ALrd120046.fits Sky Flat evening 3.5 20878 7435 32.5 34683.5
zt_ALrd120047.fits Sky Flat evening 5.0 22790 8114 2

In [16]:
print(sci_list_78[22:])   # interesting sci images in filter #78

       file       
------------------
zt_ALrd120079.fits
zt_ALrd120083.fits
zt_ALrd120087.fits
zt_ALrd120098.fits
zt_ALrd120099.fits
zt_ALrd120100.fits
zt_ALrd120113.fits
zt_ALrd120114.fits
zt_ALrd120115.fits
zt_ALrd120119.fits
zt_ALrd120120.fits
zt_ALrd120121.fits
zt_ALrd120122.fits


In [17]:
sci_list_78 = sci_list_78[22:]
print(sci_list_78)

       file       
------------------
zt_ALrd120079.fits
zt_ALrd120083.fits
zt_ALrd120087.fits
zt_ALrd120098.fits
zt_ALrd120099.fits
zt_ALrd120100.fits
zt_ALrd120113.fits
zt_ALrd120114.fits
zt_ALrd120115.fits
zt_ALrd120119.fits
zt_ALrd120120.fits
zt_ALrd120121.fits
zt_ALrd120122.fits


In [21]:
for file in sci_list_78:
    image = CCDData.read(directory / file)
    f_image = ccdproc.flat_correct(image,master_flat_78)
    name_of_file = 'f'+ str(image.header['FILENAME'])
    f_image.header['FILENAME']  = name_of_file
    f_image.header['HISTORY']   = str(datetime.datetime.now())[0:18]+' astropy ccdproc flat_correct '
    f_image.header['HISTORY']   = 'using NOT2008/N1/master_flat_78.fits master FLAT' 
    print(f'writting  {name_of_file}  in {directory}')
    f_image.write(directory / name_of_file, overwrite='yes')

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.


writting  fzt_ALrd120079.fits  in N1


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.


writting  fzt_ALrd120083.fits  in N1


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.


...

writting  fzt_ALrd120119.fits  in N1


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.


writting  fzt_ALrd120120.fits  in N1


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.


writting  fzt_ALrd120121.fits  in N1
writting  fzt_ALrd120122.fits  in N1


In [22]:
matches = (ic1.summary['ALFLTID'] == 76)
sci_list_76 = ic1.summary['file'][matches]
print(sci_list_76)

       file       
------------------
zt_ALrd120011.fits
zt_ALrd120012.fits
zt_ALrd120013.fits
zt_ALrd120014.fits
zt_ALrd120015.fits
zt_ALrd120016.fits
zt_ALrd120017.fits
zt_ALrd120018.fits
zt_ALrd120019.fits
zt_ALrd120020.fits
               ...
zt_ALrd120104.fits
zt_ALrd120105.fits
zt_ALrd120106.fits
zt_ALrd120110.fits
zt_ALrd120111.fits
zt_ALrd120112.fits
zt_ALrd120116.fits
zt_ALrd120117.fits
zt_ALrd120118.fits
zt_ALrd120122.fits
zt_ALrd120123.fits
Length = 42 rows


In [24]:
sci_images_76 = []
for file in sci_list_76:
    sci_images_76.append(CCDData.read(directory / file)) 

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.


In [25]:
print('Filename          Object        exp Mean std min  max')
exposure = []
for i in range(len(sci_list_76)):
    print(i,sci_images_76[i].header['FILENAME'], 
          sci_images_76[i].header['OBJECT'], 
          sci_images_76[i].header['EXPTIME'], 
          int(np.mean(sci_images_76[i])), 
          int(np.std(sci_images_76[i])), 
          np.min(sci_images_76[i]), 
          np.max(sci_images_76[i]))

Filename          Object        exp Mean std min  max
0 zt_ALrd120011.fits Dome Flat R 1.0 27142 1428 30.5 31854.0
1 zt_ALrd120012.fits Dome Flat R 1.0 27100 1427 49.5 31773.0
2 zt_ALrd120013.fits Dome Flat R 1.0 27155 1429 32.5 31698.0
3 zt_ALrd120014.fits Dome Flat R 1.0 27223 1433 34.5 31742.5
4 zt_ALrd120015.fits Dome Flat R 1.0 27123 1428 38.5 42562.0
5 zt_ALrd120016.fits Dome Flat R 1.0 27045 1423 33.5 31798.0
6 zt_ALrd120017.fits Dome Flat R 1.0 27142 1429 34.5 31516.5
7 zt_ALrd120018.fits Dome Flat R 1.0 27298 1437 33.5 31987.0
8 zt_ALrd120019.fits Dome Flat R 1.0 27311 1437 29.5 37468.5
9 zt_ALrd120020.fits Dome Flat R 1.0 27486 1447 48.5 39239.5
10 zt_ALrd120057.fits Sky Flat evening 2.5 25231 1442 40.5 33976.0
11 zt_ALrd120058.fits Sky Flat evening 3.299 24251 1387 28.5 38185.5
12 zt_ALrd120059.fits Sky Flat evening 4.9 25907 1485 37.5 30151.5
13 zt_ALrd120060.fits Sky Flat evening 7.0 26334 1506 45.5 35667.5
14 zt_ALrd120061.fits Sky Flat evening 10.0 26347 1608 56.5 65181.

In [27]:
print(sci_list_76[18:42])   # interesting sci images in filter #76

       file       
------------------
zt_ALrd120077.fits
zt_ALrd120078.fits
zt_ALrd120084.fits
zt_ALrd120085.fits
zt_ALrd120086.fits
zt_ALrd120089.fits
zt_ALrd120090.fits
zt_ALrd120091.fits
zt_ALrd120095.fits
zt_ALrd120096.fits
               ...
zt_ALrd120104.fits
zt_ALrd120105.fits
zt_ALrd120106.fits
zt_ALrd120110.fits
zt_ALrd120111.fits
zt_ALrd120112.fits
zt_ALrd120116.fits
zt_ALrd120117.fits
zt_ALrd120118.fits
zt_ALrd120122.fits
zt_ALrd120123.fits
Length = 24 rows


In [28]:
sci_list_76 = sci_list_76[18:42]
print(sci_list_76)

       file       
------------------
zt_ALrd120077.fits
zt_ALrd120078.fits
zt_ALrd120084.fits
zt_ALrd120085.fits
zt_ALrd120086.fits
zt_ALrd120089.fits
zt_ALrd120090.fits
zt_ALrd120091.fits
zt_ALrd120095.fits
zt_ALrd120096.fits
               ...
zt_ALrd120104.fits
zt_ALrd120105.fits
zt_ALrd120106.fits
zt_ALrd120110.fits
zt_ALrd120111.fits
zt_ALrd120112.fits
zt_ALrd120116.fits
zt_ALrd120117.fits
zt_ALrd120118.fits
zt_ALrd120122.fits
zt_ALrd120123.fits
Length = 24 rows


In [30]:
for file in sci_list_76:
    image = CCDData.read(directory / file)
    f_image = ccdproc.flat_correct(image,master_flat_76)
    name_of_file = 'f'+ str(image.header['FILENAME'])
    f_image.header['FILENAME']  = name_of_file
    f_image.header['HISTORY']   = str(datetime.datetime.now())[0:18]+' astropy ccdproc flat_correct '
    f_image.header['HISTORY']   = 'using NOT2008/N1/master_flat_76.fits master FLAT' 
    print(f'writting {name_of_file} in {directory}')
    f_image.write(directory / name_of_file, overwrite='yes')

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.


writting fzt_ALrd120077.fits in N1


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.


writting fzt_ALrd120078.fits in N1


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.


...

writting fzt_ALrd120117.fits in N1


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.


writting fzt_ALrd120118.fits in N1
writting fzt_ALrd120122.fits in N1


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.


writting fzt_ALrd120123.fits in N1


In [44]:
matches = (ic1.summary['FBFLTID'] == 49)
sci_list_49 = ic1.summary['file'][matches]
print(sci_list_49)

       file       
------------------
zt_ALrd120022.fits
zt_ALrd120033.fits
zt_ALrd120034.fits
zt_ALrd120035.fits
zt_ALrd120036.fits
zt_ALrd120037.fits
zt_ALrd120038.fits
zt_ALrd120039.fits
zt_ALrd120040.fits
zt_ALrd120041.fits
               ...
zt_ALrd120072.fits
zt_ALrd120080.fits
zt_ALrd120081.fits
zt_ALrd120082.fits
zt_ALrd120088.fits
zt_ALrd120092.fits
zt_ALrd120093.fits
zt_ALrd120094.fits
zt_ALrd120107.fits
zt_ALrd120108.fits
zt_ALrd120109.fits
Length = 32 rows


In [45]:
sci_images_49 = []
for file in sci_list_49:
    sci_images_49.append(CCDData.read(directory / file)) 

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.


In [46]:
print('Filename          Object        exp Mean std min  max')
exposure = []
for i in range(len(sci_list_49)):
    print(i,sci_images_49[i].header['FILENAME'], 
          sci_images_49[i].header['OBJECT'], 
          sci_images_49[i].header['EXPTIME'], 
          int(np.mean(sci_images_49[i])), 
          int(np.std(sci_images_49[i])), 
          np.min(sci_images_49[i]), 
          np.max(sci_images_49[i]))

Filename          Object        exp Mean std min  max
0 zt_ALrd120022.fits Dome Flat NB#49 30.0 27904 13348 28.5 45106.5
1 zt_ALrd120033.fits Dome Flat NB#49 20.0 18595 8896 27.5 32448.5
2 zt_ALrd120034.fits Dome Flat NB#49 20.0 18572 8885 30.5 29703.0
3 zt_ALrd120035.fits Dome Flat NB#49 20.0 18546 8872 18.5 31540.0
4 zt_ALrd120036.fits Dome Flat NB#49 20.0 18626 8911 25.5 31379.0
5 zt_ALrd120037.fits Dome Flat NB#49 20.0 18628 8912 22.5 37198.0
6 zt_ALrd120038.fits Dome Flat NB#49 20.0 18631 8914 11.5 34507.0
7 zt_ALrd120039.fits Dome Flat NB#49 20.0 18540 8871 5.0 30516.0
8 zt_ALrd120040.fits Dome Flat NB#49 20.0 18525 8863 29.5 29800.5
9 zt_ALrd120041.fits Dome Flat NB#49 20.0 18524 8863 27.5 29628.0
10 zt_ALrd120042.fits Dome Flat NB#49 20.0 18527 8865 17.5 29785.5
11 zt_ALrd120043.fits Dome Flat NB#49 10.0 9613 4177 -2.5 26641.0
12 zt_ALrd120051.fits Sky Flat evening 18.0 22792 9472 31.5 37293.5
13 zt_ALrd120052.fits Sky Flat evening 27.0 21157 8792 29.5 33728.0
14 zt_ALrd120053.

In [47]:
print(sci_list_49[22:32])   # interesting sci images in filter #76

       file       
------------------
zt_ALrd120080.fits
zt_ALrd120081.fits
zt_ALrd120082.fits
zt_ALrd120088.fits
zt_ALrd120092.fits
zt_ALrd120093.fits
zt_ALrd120094.fits
zt_ALrd120107.fits
zt_ALrd120108.fits
zt_ALrd120109.fits


In [48]:
sci_list_49 = sci_list_49[22:32]
print(sci_list_49)

       file       
------------------
zt_ALrd120080.fits
zt_ALrd120081.fits
zt_ALrd120082.fits
zt_ALrd120088.fits
zt_ALrd120092.fits
zt_ALrd120093.fits
zt_ALrd120094.fits
zt_ALrd120107.fits
zt_ALrd120108.fits
zt_ALrd120109.fits


In [49]:
for file in sci_list_49:
    image = CCDData.read(directory / file)
    f_image = ccdproc.flat_correct(image,master_flat_49)
    name_of_file = 'f'+ str(image.header['FILENAME'])
    f_image.header['FILENAME']  = name_of_file
    f_image.header['HISTORY']   = str(datetime.datetime.now())[0:18]+' astropy ccdproc flat_correct '
    f_image.header['HISTORY']   = 'using NOT2008/N1/master_flat_49.fits master FLAT' 
    print('writting name_of_file in directory')
    f_image.write(directory / name_of_file, overwrite='yes')

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.


writting name_of_file in directory


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.


writting name_of_file in directory


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.


...

writting name_of_file in directory


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.


writting name_of_file in directory
writting name_of_file in directory


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to    28.757278 from OBSGEO-[XYZ].
Set OBSGEO-H to     2378.994 from OBSGEO-[XYZ]'.


writting name_of_file in directory
